# 숫자형으로 표현된 범주형 데이터 처리하기

이 노트북은 특성(feature)의 실제 의미는 범주형이지만, 데이터가 숫자로 표현되어 있을 때 발생할 수 있는 문제점과 해결 방법을 다룹니다.

예를 들어, `[0, 1, 2, 1]`과 같은 데이터는 연속적인 숫자일 수도 있지만, '옷 사이즈'(0=S, 1=M, 2=L)와 같은 범주를 나타낼 수도 있습니다. 이러한 데이터를 올바르게 처리하지 않으면 모델이 잘못된 학습을 할 수 있습니다.

**학습 목표:**
1.  `pandas.get_dummies`가 숫자형 범주 데이터를 어떻게 처리하는지 이해한다.
2.  `scikit-learn.preprocessing.OneHotEncoder`를 사용하여 숫자 및 문자열 범주 데이터를 일관되게 처리하는 방법을 배운다.

### 1. 라이브러리 임포트 및 샘플 데이터 생성

In [ ]:
import pandas as pd
import numpy as np

# 샘플 데이터프레임 생성
# '숫자특성' 컬럼은 실제로는 범주를 의미하지만, 데이터 타입은 숫자(int)입니다.
demo_df = pd.DataFrame({ 
    '숫자특성':[0, 1, 2, 1],  
    '범주형특성': ['양말', '여우', '양말', '상자']    
})

print("원본 데이터프레임:")
print(demo_df)
print("
데이터 타입:")
print(demo_df.dtypes)

### 2. `pandas.get_dummies` 사용 시 문제점

`get_dummies` 함수는 기본적으로 **문자열(object)이나 카테고리(category) 타입**의 컬럼에 대해서만 원-핫 인코딩을 수행합니다. 숫자(int, float) 타입의 컬럼은 연속형 수치 데이터로 간주하여 변환하지 않습니다.

In [ ]:
# get_dummies를 그냥 적용한 경우
df1 = pd.get_dummies(demo_df)

print("get_dummies 처리 결과:")
print(df1)
# 결과: '숫자특성'은 변환되지 않고, '범주형특성'만 원-핫 인코딩되었습니다.

#### 해결 방법: 데이터 타입 변경

`get_dummies`가 '숫자특성'을 범주형으로 인식하게 하려면, 해당 컬럼의 데이터 타입을 `str` (문자열) 또는 `category`로 명시적으로 변경해야 합니다.

In [ ]:
# '숫자특성' 컬럼의 타입을 문자열로 변경
demo_df_fixed = demo_df.copy() # 원본 데이터 보존을 위해 복사
demo_df_fixed['숫자특성'] = demo_df_fixed['숫자특성'].astype(str)

print("타입 변경 후 데이터 타입:")
print(demo_df_fixed.dtypes)

# 다시 get_dummies 적용
df2 = pd.get_dummies(demo_df_fixed)

print("
타입 변경 후 get_dummies 처리 결과:")
print(df2)
# 결과: 이제 '숫자특성'도 올바르게 원-핫 인코딩되었습니다.

### 3. `scikit-learn.preprocessing.OneHotEncoder` 사용

`scikit-learn`의 `OneHotEncoder`는 데이터 타입을 사전에 변경할 필요 없이, 숫자형과 문자열 범주 데이터를 모두 일관되게 처리할 수 있어 더 편리하고 강력합니다. 머신러닝 파이프라인에 통합하기에도 용이합니다.

- `sparse_output=False`: 결과를 일반적인 `numpy` 배열로 반환하도록 설정합니다. (기본값은 `True`이며, 메모리 효율적인 희소 행렬(sparse matrix)로 반환)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# OneHotEncoder 객체 생성
ohe = OneHotEncoder(sparse_output=False)

# 원본 데이터프레임(demo_df)을 사용하여 학습하고 변환합니다.
# fit_transform은 numpy 배열을 반환합니다.
encoded_data = ohe.fit_transform(demo_df)

# 생성된 특성(컬럼) 이름 확인
feature_names = ohe.get_feature_names_out()
print("OneHotEncoder가 생성한 특성 이름:")
print(feature_names)

# 변환된 numpy 배열 확인
print("
변환된 데이터 (Numpy 배열):")
print(encoded_data)

# 보기 쉽게 데이터프레임으로 변환
encoded_df = pd.DataFrame(encoded_data, columns=feature_names)
print("
변환된 데이터를 DataFrame으로 표시:")
print(encoded_df)